In [32]:
import pickle
import numpy as np
import matplotlib.pylab as plt
import uproot
from sklearn.metrics import confusion_matrix

In [33]:
def return_accuracy(predict1,predict2,validation):
    mass_predict1 = []
    for i in predict1:
        if i <2.5:
            mass_predict1.append(1)
        else:
            mass_predict1.append(4)
    cm=confusion_matrix(validation,mass_predict1)

    mass_predict4 = []
    for i in predict2:
        if i <2.5:
            mass_predict4.append(1)
        else:
            mass_predict4.append(4)
    cm1=confusion_matrix(validation,mass_predict4)
    
    return cm[0][0]/np.sum(cm[0]),cm1[1][1]/np.sum(cm1[1]),np.array(mass_predict1),np.array(mass_predict4)

In [34]:
file = uproot.open('/home/andy/12632.root')
file2 = uproot.open('/home/andy/12633.root')

In [35]:
model1 =  pickle.load((open('model.sav','rb')))

In [36]:
S125_1 = file['LaputopParams']['s125'].array()
S125_2 = file2['LaputopParams']['s125'].array()
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
energy1 = file['MCPrimary']['energy'].array()
energy2 = file2['MCPrimary']['energy'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()

slc_charge1 = file['LaputopSLCPE']['charge'].array()
slc_charge2 = file2['LaputopSLCPE']['charge'].array()

slc_charge1 = [np.sum(i) for i in slc_charge1]
slc_charge2 = [np.sum(i) for i in slc_charge2]

hlc_charge1 = file['LaputopHLCPE']['charge'].array()
hlc_charge2 = file2['LaputopHLCPE']['charge'].array()

hlc_charge1 = [np.sum(i) for i in hlc_charge1]
hlc_charge2 = [np.sum(i) for i in hlc_charge2]

Qtot1 = [i+j for i,j in zip(slc_charge1,hlc_charge1)]
Qtot2 = [i+j for i,j in zip(slc_charge2,hlc_charge2)]

mass1 = [1 for i in range(len(Qtot1))]
mass2 = [4 for i in range(len(Qtot2))]

red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()

In [37]:
A = np.append(A1,A2)
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.append(S125_1,S125_2)
beta = np.append(beta1,beta2)
chi2 = np.append(chi2_1,chi2_2)
zenith = np.append(zenith1,zenith2)
zenith_new = np.append(zenith1,zenith2)
mass = np.append(mass1,mass2)
slc_charge = np.append(slc_charge1,slc_charge2)
hlc_charge = np.append(hlc_charge1,hlc_charge2)
qtot = np.append(Qtot1,Qtot2)

energy = np.append(energy1,energy2)
red = np.append(red1,red2)

In [38]:
A = A[np.isfinite(chi2)]
D = D[np.isfinite(chi2)]
S125 = np.log10(S125[np.isfinite(chi2)])
beta = np.log10(beta[np.isfinite(chi2)])
chi2_new = chi2[np.isfinite(chi2)]
zenith = np.cos(zenith[np.isfinite(chi2)])
mass = mass[np.isfinite(chi2)]
energy = np.log10(energy[np.isfinite(chi2)])
slc_charge = np.array(slc_charge)[np.isfinite(chi2)]
hlc_charge = np.array(hlc_charge)[np.isfinite(chi2)]
qtot = np.array(qtot)[np.isfinite(chi2)]
zenith_new = np.array(zenith_new)[np.isfinite(chi2)]
red = np.array(red)[np.isfinite(chi2)]

In [39]:
input_variable2_old = np.array(list(zip(A,slc_charge,beta,chi2_new)))[chi2_new<5]
output_new_old = np.array(mass)[chi2_new<5][chi2_new<5]
energy_new = np.array(energy)[chi2_new<5][chi2_new<5]

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
prediction1 = model1.predict(input_variable2_old)

In [ ]:
data1 = np.array(list(zip(input_variable2_old,output_new_old)))
data2 = np.array(energy_new)
array_split = np.array_split(sorted(list(zip(data1,data2)),key=lambda x:x[1]),20)

In [ ]:
x = []
y_proton = []
y_iron = []
for i in array_split:
    energy = list(zip(*i))[1]
    mass = list(zip(*list(zip(*i))[0]))[1]
    data = list(zip(*list(zip(*i))[0]))[0]
    prediction1 = model1.predict(data)
    #prediction2 = model2.predict(data)
    #prediction1 = (prediction1+prediction2)/2
    output_prediction = []
    for i in prediction1:
        if i < 2.5:
            output_prediction.append(1)
        else:
            output_prediction.append(4)
    
    cm = confusion_matrix(output_prediction,mass)
    x.append((max(energy)+min(energy))/2)
    y_proton.append(cm[0][0]/np.sum(cm[0]))
    y_iron.append(cm[1][1]/np.sum(cm[1]))

In [ ]:
plt.scatter(x,y_proton,label='proton')
plt.scatter(x,y_iron,label='iron')
plt.legend()
plt.show()

In [ ]:
print(y_iron,np.mean(y_iron))

In [ ]:
print(y_proton,np.mean(y_proton))

In [ ]:
print(np.mean(y_iron)/2+np.mean(y_proton)/2)

In [ ]:
plt.hist(red[red<300],bins=100)
plt.show()